# E3 - Parallellisering

## E3.1 Parallellprogrammering i Python

* Ett vanlig Python-program kjører i en tråd i en prosess og kan bare gjøre en ting om gangen
* Flere tråder og prosesser kan opprettes for å gjøre ting i parallell
  * Begge har fordeler og ulemper

## E3.2 Subprocess

* Python modul for å starte og kommunisere med nye prosesser
* Blir ofte brukt for å starte andre programmer fra Python kode
  * Kan lese og gjøre noe med output fra programmet
  * Kan gi input til programmet under kjøring

__subprocess.run()__
* Brukes for å starte en prosess
* __Venter__ til prosessen er ferdig, og returnerer ett CompletedProcess objekt
  * Innholdet avhenger av hvilke argumenter som ble gitt til subprocess.run()

In [ ]:
import subprocess
out = subprocess.run("dir", shell=True, capture_output=True, text=True)
print(out)
print()
print(out.stdout)

__subprocess.Popen__
* Er mer fleksibel, men mer tungvinn å bruke
 * Må ikke vente til prosessen avsluttes (men kan)

In [ ]:
import subprocess
proc = subprocess.Popen("sleep 5", shell=True, stdout=subprocess.PIPE, text=True)
print("Returncode:", proc.poll())

try:
    proc.wait(5)
except subprocess.TimeoutExpired:
    proc.kill()
    print("Process took to long...")

print(proc.stdout.read())

## E3.3 Multithreading

* Opprette tråder i programmet for å gjøre ting i "parallell"
* Global Interpreter Lock (GIL)
  * Kun en tråd kjører om gangen, selv på systemer med flere prosessorer
  * OSet velger hvilken tråd som kjører, og veksler mellom dem
* Nyttig i program hvor man venter på andre ting
  * Databaser, web-requests etc...
 
__Enkleste metode:__ 
```python
threading.Thread(target=enFunksjon, args=argumenter)
```


__Mer komplisert metode:__
```python
class myThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        ...
    def run(self):
        ...
```

In [ ]:
import threading
import time

def print_time(threadName, delay):
    for i in range(5):
        time.sleep(delay)
        print(f"{threadName}: {time.time()}")

t1 = threading.Thread(target=print_time, args=("Thread-1", 2))
t2 = threading.Thread(target=print_time, args=("Thread-2", 4))

t1.start()
t2.start()

In [ ]:
import threading
import time

def process_stuff(start, stop):
    a = 0
    for i in range(start, stop):
        a += i**i
    return a

s = time.time()
t1 = threading.Thread(target=process_stuff, args=(0, 10000))
t1.start()
t1.join()
print(f"Processing time: {time.time() - s} seconds")

In [ ]:
import threading
import time

def process_stuff(name, start, stop):
    print(f"{name} started!")
    a = 0
    for i in range(start, stop):
        a += i**i
    print(f"{name} stopped!")
    return a

threads = []
calc_start = 0
calc_stop = 10000
num_threads = 2
chunk_size = (calc_stop - calc_start) // num_threads

s = time.time()
for i in range(num_threads):
    t = threading.Thread(
        target=process_stuff, args=(f"Thread-{i}", calc_start + chunk_size*i, calc_start + chunk_size*(i+1))
    )
    t.start()
    threads.append(t)

for t in threads:
    t.join()
print(f"Processing time: {time.time() - s} seconds")

## E3.4 Multiprocessing

* Dupliserer _hele_ Python-prosessen
* Kjører kode i flere prosesser for ekte parallellisering
  * Kjører på flere prosessorkjerner samtidig
* Nyttig for å gjøre tung prosessering
* Samme API som multithreading:

__Enkleste metode:__ 
```python
multiprocessing.Process(target=enFunksjon, args=argumenter)
```

__Mer komplisert metode:__
```python
class myProcess(multiprocessing.Process):
    def __init__(self):
        multiprocessing.Process.__init__(self)
        ...
    def run(self):
        ...
```

In [ ]:
import multiprocessing
import time

def process_stuff(name, start, stop):
    print(f"{name} started!")
    a = 0
    for i in range(start, stop):
        a += i**i
    print(f"{name} stopped!")
    return a

if __name__ == '__main__':
    processes = []
    calc_start = 0
    calc_stop = 10000
    num_procs = 1
    chunk_size = (calc_stop - calc_start) // num_procs

    s = time.time()
    for i in range(num_procs):
        p = multiprocessing.Process(
            target=process_stuff, args=(f"Proc-{i}", calc_start + chunk_size*i, calc_start + chunk_size*(i+1))
        )
        p.start()
        processes.append(p)

    for p in processes:
        p.join()
    print(f"Processing time: {time.time() - s} seconds")

## E3.5 Queues

* Toveis kommunikasjon mellom prosesser/tråder
* For tråder:
  * _queue.Queue()_
  * Sender Python-objekter i minnet
  * Alt skjer i samme prosess
* For prosesser:
  * _multiprocessing.Queue()_
  * Må konvertere objekter til bytes før de sendes
  * Kan derfor være litt problematisk å bruke med kompliserte objekter
  * Er ikke anbefalt å bruke køer til å sende store mengder data
* Kan bruke .join() på køer
  * Må da bruke .task_done() for hvert element som hentes ut av køen
  * Holder en teller over antallet elementer som puttes i køen med .put()

In [ ]:
import threading
import queue
import time

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        print(f'Working on {item}')
        time.sleep(1)
        print(f'Finished {item}')
        q.task_done()

# Start worker tråd
threading.Thread(target=worker, daemon=True).start()

# Send oppgaver til worker tråden via køen
for item in range(5):
    q.put(item)
print('All task requests sent\n', end='')

# Vent til alle oppgaver er ferdig utført
q.join()
print('All work completed')

__Mer komplisert eksempel:__

In [ ]:
import threading
import subprocess
import queue

def executor(cmd_queue, res_queue):
    while True:
        cmd = cmd_queue.get()
        if cmd == "!quit":
            break
        out = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        res_queue.put(out.stdout)

cq = queue.Queue()
rq = queue.Queue()
threading.Thread(target=executor, args=(cq,rq)).start()

cmd = ""
while True:
    cmd = input("Input command: ")
    cq.put(cmd)
    if cmd == "!quit":
        print("Quitting!")
        break
    res = rq.get()
    print("Got:", res)

# E3.6 Oppgaver

## Oppgave: subprocess
* Lag ett script som:
  * Tar en cmd-kommando som input fra brukeren
  * Eksekverer kommandoen i en subprocess
  * printer resultatet av kommandoen
  * looper til toppen
  * Bonus: printer en punktum "." hvert sekund til kommandoen er ferdig

## Oppgave: Multithreading
* Konverter scriptet under til å bruke tråder slik at det går fortere

In [ ]:
import requests
import time
import os

def download_url(url, path):
    res = requests.get(url)
    with open(path, "wb") as f:
        f.write(res.content)
    
urls = [
    ("http://nrk.no", "pages/nrk.no.txt"),
    ("http://vg.no", "pages/vg.no.txt"),
    ("http://dagbladet.no", "pages/dagbladet.no.txt"),
    ("http://python.org", "pages/python.org.txt"),
] * 20 # <-- Litt juks for å forenkle eksempelet
    
if __name__=='__main__':
    try:
        os.mkdir("pages")
    except:
        pass
    
    print("Started downloading...")
    s = time.time()
    for url, path in urls:
        download_url(url, path)
    print(f"Download time: {time.time() - s} seconds")

## Oppgave: Multiprocessing
* Funksjonen under representerer en prosesserings-tung oppgave
* Målet er å få runksjonen til å returnere så fort som mulig
* Bruk multiprocessing til å løse oppgaven fortere

```python
import random

def oppgave():
    mål = 1337
    while True:
        if random.randint(0, 10000) == mål:
            print("Done!")
            break
```

## Oppgave: Queues
* Gå tilbake til oppgave 'Oppgave: Multithreading'
* Konverter scriptet til å bruke worker-threads
* Start 5 tråder som alle venter på oppgaver, og signaliserer når de er ferdige
* Dette er den anbefalte måten å lage denne typen script på, da man ofte får for mange tråder dersom man oppretter en per. oppgave.
  * De fleste OS begrenser antallet tråder hver prosess kan ha
  * I tillegg er det litt overhead med å opprette tråder og veksle mellom dem